In [2]:
!pip install flask transformers torch
!pip install --upgrade transformers



In [4]:
!pip install --upgrade accelerate

In [3]:
!pip install transformers datasets evaluate scikit-learn


In [1]:
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)
classifier = pipeline("sentiment-analysis")

@app.route("/analyze", methods=["POST"])
def analyze():
    text = request.json["text"]
    result = classifier(text)[0]
    return jsonify(result)

app.run(port=5000)


KeyboardInterrupt: 

In [6]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
print(classifier("I love pizza"))


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
partially initialized module 'torch._dynamo' has no attribute 'decorators' (most likely due to a circular import)

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch

# Load GoEmotions and tokenizer
dataset = load_dataset("go_emotions")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Reward categories and mappings
label_mapping = {
    "joy": "self-care",
    "admiration": "gratitude",
    "gratitude": "social",
    "pride": "productive",
    "relief": "self-care",
    "love": "social",
    "excitement": "self-care",
    "amusement": "self-care",
    "caring": "social",
    "remorse": "social",
    "desire": "self-care",
    "approval": "social",
    "realization": "self-care",
    "curiosity": "productive",
    "optimism": "self-care",
}

reward_labels = ["productive", "self-care", "gratitude", "social", "neutral"]
label2id = {label: i for i, label in enumerate(reward_labels)}
id2label = {v: k for k, v in label2id.items()}
num_labels = len(label2id)

label_feature = dataset["train"].features["labels"].feature

# Step 1: Map emotion labels to reward categories
def map_labels(example):
    matched_labels = set()
    for label_id in example["labels"]:
        label_name = label_feature.int2str(label_id)
        if label_name in label_mapping:
            matched_labels.add(label_mapping[label_name])
    if not matched_labels:
        matched_labels = {"neutral"}
    return {"reward_labels": list(matched_labels)}

# Step 2: Filter only those with at least one valid reward label
def valid_labels(example):
    return any(label in reward_labels for label in example["reward_labels"])

# Step 3: Encode to multi-hot format
def encode_labels(example):
    multi_hot = [0] * len(label2id)
    for label in example["reward_labels"]:
        if label in label2id:
            multi_hot[label2id[label]] = 1
    return {"labels": multi_hot}

# Step 4: Tokenize the text
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# ---- Apply pipeline ----
# Apply mapping first (keep text column)
mapped_dataset = dataset["train"].map(map_labels)

# Filter invalid examples
filtered_dataset = mapped_dataset.filter(valid_labels)

# Tokenize (this will keep the text column)
tokenized_dataset = filtered_dataset.map(tokenize)

# Encode labels (this will add the multi-hot labels)
encoded_dataset = tokenized_dataset.map(encode_labels)

encoded_dataset = encoded_dataset.map(
    lambda x: {"labels": torch.tensor(x["labels"], dtype=torch.float)},
    batched=True
)

# Remove unnecessary columns (only keep what's needed for training)
final_dataset = encoded_dataset.remove_columns(["id", "reward_labels"])

def encode_labels(example):
    multi_hot = [0] * len(label2id)
    for label in example["reward_labels"]:
        if label in label2id:
            multi_hot[label2id[label]] = 1
    return {"labels": torch.tensor(multi_hot, dtype=torch.float)}  # Changed to float

# Set format for PyTorch
final_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Step 5: Train/val split
split_dataset = final_dataset.train_test_split(test_size=0.1)

In [ ]:
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from transformers import (
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding,
    AutoTokenizer
)
from datasets import load_dataset

# 1. Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = (logits > 0).astype(int)
    return {
        "f1_micro": f1_score(labels, predictions, average="micro"),
        "f1_macro": f1_score(labels, predictions, average="macro"),
        "accuracy": accuracy_score(labels, predictions),
    }

# 2. Load and prepare dataset
dataset = load_dataset("go_emotions")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Sample small subset
small_dataset = dataset["train"].train_test_split(test_size=0.95, seed=42)["train"]

# Label configuration
label_mapping = {
    "joy": "self-care",
    "admiration": "gratitude",
    "gratitude": "social",
    "pride": "productive",
    "relief": "self-care",
    "love": "social",
    "excitement": "self-care",
    "amusement": "self-care",
    "caring": "social",
    "remorse": "social",
    "desire": "self-care",
    "approval": "social",
    "realization": "self-care",
    "curiosity": "productive",
    "optimism": "self-care",
}

reward_labels = ["productive", "self-care", "gratitude", "social", "neutral"]
label2id = {label: i for i, label in enumerate(reward_labels)}
id2label = {i: label for i, label in enumerate(reward_labels)}
num_labels = len(reward_labels)

# Preprocessing functions
def map_labels(example):
    matched_labels = set()
    for label_id in example["labels"]:
        label_name = dataset["train"].features["labels"].feature.int2str(label_id)
        if label_name in label_mapping:
            matched_labels.add(label_mapping[label_name])
    if not matched_labels:
        matched_labels = {"neutral"}
    return {"reward_labels": list(matched_labels)}

def encode_labels(example):
    multi_hot = [0] * num_labels
    for label in example["reward_labels"]:
        if label in label2id:
            multi_hot[label2id[label]] = 1
    return {"labels": torch.tensor(multi_hot, dtype=torch.float32)}  # Critical fix: float32 labels

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

# Apply preprocessing
small_dataset = small_dataset.map(map_labels)
small_dataset = small_dataset.filter(lambda x: any(label in reward_labels for label in x["reward_labels"]))
small_dataset = small_dataset.map(tokenize)
small_dataset = small_dataset.map(encode_labels)
small_dataset = small_dataset.remove_columns(["id", "reward_labels", "text"])

# Split into train/test
split_dataset = small_dataset.train_test_split(test_size=0.2)

# 3. Model setup
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
).to("cpu")

# 4. Training setup - Custom collator to ensure proper types
class FloatLabelCollator(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        if "labels" in batch:
            batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float32)  # Force float32
        return batch

data_collator = FloatLabelCollator(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./small-model",
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=1,
    no_cuda=True,
    dataloader_num_workers=1,
    logging_steps=10,
)

# 5. Create and run trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print("Starting training...")
print(f"Training samples: {len(split_dataset['train'])}")
print(f"Evaluation samples: {len(split_dataset['test'])}")
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting training...
Training samples: 1736
Evaluation samples: 434


In [29]:
from flask import Flask, jsonify
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

app = Flask(__name__)

# Load model and tokenizer once at startup
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Mapping from model outputs to nominal labels
id2label = {0: "Negative", 1: "Neutral", 2: "Positive"}  # Example

@app.route("/predict", methods=["GET"])
def predict_from_file():
    try:
        with open("user_input.txt", "r") as f:
            text = f.read().strip()

        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            logits = model(**inputs).logits

        prediction = torch.argmax(logits, dim=1).item()
        nominal_value = id2label.get(prediction, "Unknown")

        return jsonify({"text": text, "nominal_value": nominal_value})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=True)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\HP\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [26]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Reliable loading - will always work
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Proceed with training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    compute_metrics=compute_metrics
)
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'training_args' is not defined

In [17]:
import torch
assert torch.cuda.is_available(), "GPU not detected!"
print(f"🎉 Ready to train on: {torch.cuda.get_device_name(0)}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")

AssertionError: GPU not detected!